In [1]:
import numpy as np
import copy

In [ ]:
def relu(x):
    return max(x, 0)


class NodeGene:
    def __init__(self, id, type, h=0, bias = np.random.uniform(-1,1)):
        self.h = h
        self.type = type
        self.id = id
        self.bias = bias


class ConnectionGene:
    def __init__(self, inputNode, outputNode, weight, innovation, enabled = True):
        self.enabled = enabled
        self.innovation = innovation
        self.weight = weight
        self.outputNode = outputNode
        self.inputNode = inputNode


class Specimen:
    # generating Phenotype from Genotype
    def __init__(self, input, output):
        self.id_of_next = output + input

        inputs = [NodeGene(i, 'input') for i in range(input)]
        outputs = [NodeGene(i, 'output', h=1) for i in range(output)]

        self.nodes = inputs + outputs

        self.connections = []
        innovation = 0
        for i in range(input):
            for j in range(input, input + output):
                self.connections.append(ConnectionGene(i,j, np.random.normal(0,1), innovation))
                innovation += 1

    def forward(self, input):

        order = [node.h for node in self.nodes]
        order = np.argsort(order)
        values = [0] * len(self.nodes)

        # now we are sending the signal
        for node in [self.node[i] for i in order]:
            if node.h == 0:
                values[node.id] = input[node.id]
                for connection in self.connections:
                    if connection.inputNode == node.id:
                        values[connection.outputNode] += values[node.id] * connection.weight

            elif node.h > 0 and node.h < 1:
                activation_val = relu(values[node.id])
                for connection in self.connections:
                    values[connection.outputNode] += activation_val * connection.weight + node.bias

            elif node.h == 1:
                # this is output neuron
                output = values[node.id]

        return output


    def mutate_weight(self, threshold = 0.8):

        for connection in self.connections:
            if np.random.uniform(0,1) < threshold:
                connection.weight *= np.random.uniform(-2.5, 2.5)
            else:
                connection.weight = np.random.uniform(-1,1)

    def mutate_bias(self, threshold = 0.8):

        for node in self.nodes:
            if np.random.uniform(0,1) < threshold:
                node.bias *= np.random.uniform(-2.5, 2.5)
            else:
                node.bias = np.random.uniform(-1,1)

    def accuracy(self, x, y):

        res = []
        for inp in x:
            res.append(self.forward(inp))

        return ((res == y) * 1).mean()

    def mae(self, x, y):
        res = []
        for inp in x:
            res.append(self.forward(inp))

        return np.abs(x - y).mean()


class NEAT:

    def __init__(self, population_number, input, output):
        self.population_number =  population_number
        self.input = input
        self.output = output
        self.innovation = input * output
        self.connection_innovation = []
        self.gen_mean = []

    def start(self):
        # initialize the population
        self.population = [Specimen(self.input, self.output) for i in range(self.population_number)]
        pass
    # TODO Implement Rest





W tym tygodniu w związku z nakładem materiału teoretycznego i prototypowania nie udało mi się zrobić pełnej sieci. W przyszłym tygodniu planuję odrobić straty.

